## Import Libraries

In [16]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

## Load Model, Encoders and Scalar

In [17]:
model=load_model('model.h5')

with open('geo.pkl','rb') as file:
    geo = pickle.load(file)

with open('gender.pkl','rb') as file:
    gender = pickle.load(file)

with open('scalar.pkl','rb') as file:
    scalar = pickle.load(file)

## Input Data for Prediction

In [18]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',	
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [19]:
# Convert Input data into DataFrame
df_input = pd.DataFrame([input_data])
df_input

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


#### Convert Categorical data

In [20]:
df_input['Gender']=gender.transform(df_input['Gender'])
df_input

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [21]:
df_input_geo = pd.get_dummies(df_input['Geography'], sparse=False, dtype='int')

# Align the input dummy variables with the original dummy variables
df_input_geo = df_input_geo.reindex(columns=geo.columns, fill_value=0)
df_input_geo

,France,Germany,Spain
0,1,0,0


In [22]:
# Concate df_input_geo with df_input
df_input.drop(['Geography'],axis=1,inplace=True)
df_input = pd.concat([df_input, df_input_geo], axis=1)
df_input

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,600,1,40,3,60000,2,1,1,50000,1,0,0


## Feature Scaling

In [23]:
df_input=scalar.transform(df_input)
df_input

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

## Prediction

In [24]:
prediction = model.predict(df_input)
prediction

1/1 [==============================] - 0s 168ms/step


array([[0.05085266]], dtype=float32)

In [25]:
prediction_proba = prediction[0][0]
prediction_proba

0.050852656

In [26]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
